<a href="https://colab.research.google.com/github/UnpackJungHo/XRSimulator_Osaka/blob/dataProcesiing/DataProcessing_241223.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import datetime

# 기본 API URL과 인증 키 설정
base_url = "https://apihub.kma.go.kr/api/typ01/url/kma_sfctm3.php"
auth_key = "wnNFMnRfQYCzRTJ0X3GATw"

# 시작 및 종료 날짜 설정
start_date = datetime.datetime(2021, 1, 14)
end_date = datetime.datetime(2021, 6, 14)

# 결과를 저장할 리스트
all_data = []

# 월별로 데이터를 요청
current_date = start_date
while current_date < end_date:
    next_month = current_date + datetime.timedelta(days=30)  # 다음 달로 이동
    if next_month > end_date:
        next_month = end_date  # 마지막 범위 설정

    # 요청 URL 생성
    url = f"{base_url}?tm1={current_date.strftime('%Y%m%d%H%M')}&tm2={next_month.strftime('%Y%m%d%H%M')}&stn=108&authKey={auth_key}"
    print(f"Requesting data from {current_date} to {next_month}")

    # API 요청
    response = requests.get(url)

    # JSON 형식인지 확인 후 데이터 추가
    if 'application/json' in response.headers.get('Content-Type', ''):
        json_response = response.json()
        all_data.append(json_response)
    else:
        print("Response is not in JSON format. Content-Type:", response.headers.get('Content-Type'))
        print("Response content:", response.text)

    # 현재 날짜를 다음 달로 갱신
    current_date = next_month

# 결과 출력
print("모든 데이터를 성공적으로 가져왔습니다!")
print(all_data)


In [ ]:
import requests
import datetime
import sys

def main():
    # 기본 API URL과 인증 키 설정
    base_url = "https://apihub.kma.go.kr/api/typ01/url/kma_sfctm3.php"
    auth_key = "wnNFMnRfQYCzRTJ0X3GATw"  # 필요하면 실제 키로 교체

    # 시작 및 종료 날짜 설정
    start_date = datetime.datetime(2020, 1, 1)
    end_date = datetime.datetime(2020, 6, 15)

    # 월별로 데이터를 요청
    current_date = start_date
    all_text = ""

    while current_date < end_date:
        # 다음 달로 이동(대략 30일)
        next_month = current_date + datetime.timedelta(days=30)
        if next_month > end_date:
            next_month = end_date

        # URL 생성
        url = (
            f"{base_url}?"
            f"tm1={current_date.strftime('%Y%m%d%H%M')}&"
            f"tm2={next_month.strftime('%Y%m%d%H%M')}&"
            f"stn=108&"
            f"authKey={auth_key}"
        )
        print(f"Requesting data from {current_date} to {next_month}")

        # API 요청
        response = requests.get(url)

        # 응답이 JSON 형태인지 확인하고, 아니면 텍스트로 누적
        if 'application/json' in response.headers.get('Content-Type', ''):
            # JSON 형태면 pass
            pass
        else:
            all_text += response.text + "\n"

        current_date = next_month

    # 한 줄씩 분리
    lines = all_text.splitlines()

    # 필요한 컬럼명 설정
    target_columns = ["STN", "TA", "RN", "INT", "DAY", "WW", "WC", "WP",
                      "HR3", "TOT", "TOP", "LOW", "WH", "WD", "HM", "CT"]

    # 헤더 라인 찾기
    header_line1 = None
    header_line2 = None
    for i, line in enumerate(lines):
        if line.startswith('# YYMMDDHHMI'):
            header_line1 = line
            header_line2 = lines[i+1]
            break

    if not header_line1 or not header_line2:
        print("헤더 라인을 찾지 못했습니다.")
        sys.exit(0)

    # 헤더 라인 파싱
    header_cols1 = header_line1.strip('# ').split()
    header_cols2 = header_line2.strip('# ').split()

    # 컬럼 인덱스 매핑 초기화
    col_index_map = {}

    # 특수 컬럼들의 위치 찾기
    for i, (col1, col2) in enumerate(zip(header_cols1, header_cols2)):
        # CA TOT 찾기
        if col1 == "CA" and col2 == "TOT":
            col_index_map["TOT"] = i
        # CT TOP 찾기
        elif col1 == "CT" and col2 == "TOP":
            col_index_map["TOP"] = i
        # CT LOW 찾기
        elif col1 == "CT" and col2 == "LOW":
            col_index_map["LOW"] = i
        # RN INT 찾기
        elif col1 == "RN" and col2 == "INT":
            col_index_map["INT"] = i
        # RN DAY 찾기
        elif col1 == "RN" and col2 == "DAY":
            col_index_map["DAY"] = i
        # SD HR3 찾기
        elif col1 == "SD" and col2 == "HR3":
            col_index_map["HR3"] = i
        # SD DAY 찾기
        elif col1 == "SD" and col2 == "DAY":
            col_index_map["SD_DAY"] = i
        # SD TOT 찾기
        elif col1 == "SD" and col2 == "TOT":
            col_index_map["SD_TOT"] = i

    # 나머지 일반 컬럼들의 인덱스 찾기
    for col in ["STN", "TA", "RN", "WW", "WC", "WP", "WH", "WD", "WS", "HM", "CT"]:
        if col in header_cols1:
            col_index_map[col] = header_cols1.index(col)

    # 최종 출력할 컬럼 헤더 정의
    columns = [
        "DateTime(YYYYMMDDHHMI)", "STN", "TA", "RN", "RN_INT", "RN_DAY", "WW", "WC",
        "WP", "SD_DAY", "SD_TOT", "SD_HR3", "CA_TOT", "CT_TOP", "CT_LOW",
        "WH", "WD", "WS", "HM", "CT"
    ]

    # 각 컬럼의 너비 설정
    widths = {
        "DateTime(YYYYMMDDHHMI)": 20,
        "STN": 6,
        "TA": 8,
        "RN": 8,
        "RN_INT": 8,
        "RN_DAY": 8,
        "WW": 6,
        "WC": 6,
        "WP": 6,
        "SD_DAY": 8,
        "SD_TOT": 8,
        "SD_HR3": 8,
        "CA_TOT": 8,
        "CT_TOP": 8,
        "CT_LOW": 8,
        "WH": 8,
        "WD": 6,
        "WS": 6,
        "HM": 8,
        "CT": 8
    }

    # 헤더 출력
    header = " ".join(f"{col:^{widths[col]}}" for col in columns)

    # 헤더 출력 (공백 구분 대신 콤마 구분으로 변경)
    print(",".join(columns))

    # 실제 데이터 출력
    for line in lines:
        # 주석 라인은 제외
        if line.startswith('#'):
            continue

        parts = line.split()
        # 날짜·시간 형태(12자리)를 갖고 있는지 검사
        if len(parts) > 1 and parts[0].isdigit() and len(parts[0]) == 12:
            dt = parts[0]
            values = [dt]

            # 출력 순서
            columns_order = [
                "STN", "TA", "RN", "INT", "DAY", "WW", "WC", "WP",
                "SD_DAY", "SD_TOT", "HR3", "TOT", "TOP", "LOW",
                "WH", "WD", "WS", "HM", "CT"
            ]

            for col in columns_order:
                if col in col_index_map:
                    values.append(parts[col_index_map[col]])
                else:
                    # 없는 경우 NA로 처리
                    values.append("NA")

            # 콤마로 구분된 문자열로 출력
            print(",".join(values))

if __name__ == "__main__":
    main()

# 새 섹션